<a href="https://colab.research.google.com/github/1o-o1/Duplication-in-Redun/blob/main/Deduplication_in_Redundant_Server.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pycrypto
!pip3 install hungarian-algorithm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 446 kB 7.9 MB/s 
  Created wheel for pycrypto: filename=pycrypto-2.6.1-cp37-cp37m-linux_x86_64.whl size=499932 sha256=36e379fb4682f397c6a8ea913284b4f3e4e32ca0336b3930c93cb616331b8546
  Stored in directory: /root/.cache/pip/wheels/cf/85/ba/bbd7c96add459de7598fb424e5ff2309baf2095c844ac0f191
Successfully built pycrypto
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import numpy as np
import pandas as pd
from hashlib import sha256
from hashlib import sha224
import string
import random
import math
import time
import base64
import hashlib
from Crypto import Random
from Crypto.Cipher import AES
from hungarian_algorithm import algorithm

In [3]:
class data_server:
  def __init__(self,id,size):
    self.id=id
    self.size=size
    self.available=True
    self.memory=np.empty(size, dtype=object)

  def availablity(self,val):
    self.available=val

  def read(self, address, lenth):
      if self.available == False:
        return None
      data=[]
      for i in range(lenth):
        data.append(self.memory[address+i])
      return data

  def write(self, address,data ):
    self.memory[address]=data

  def read_multiple(self, addresses):
    data=[]
    for address in addresses:
      data.append(self.read(address,1))
    return data

  def write_multiple(self, addresses,datas):
    for i in range(len(datas)):
      self.write(addresses[i],datas[i])
      print(datas[i])

In [4]:
class index_server:
  def __init__(self,id,data_server):
    self.id=id
    self.users = {} #contains lots of files (memory location)
    self.servers={} #contains lots of servers and each server mem location has boolean value ,used or not used
    self.hash_map ={}   #contains hash as key with memory location
    self.data_server = data_server
    self.block_size=64
    self.max_server=10


#############################################################
#                           USER                            #
#                         Functions                         #
#############################################################
  def create_user(self):
    return
  def Authenticate_user(self):
    return
  def Authorize_user(self):
    return

#check user or file exist other wise add that
# assumption user_ID and file_ID are unique
  def add_user_file(self,user_id,file_id):
    if self.users.get(user_id)==None or self.users[user_id].get(file_id)==None :
      self.users[user_id]={file_id:[]}
    else:
      return 0;

#check if a hash exist or add it
  def add_user_file_block(self,user_id,file_id,hash):
    if hash not in self.users[user_id].get(file_id):
      self.users[user_id][file_id].append(hash)
    else:
      return 0


  def delete_user_file(self):
    return;
  def delete_user_file_block(self):
    return;
#"""User"""


#############################################################
#                          SERVER                           #
#                         Functions                         #
#############################################################

#intilize the server with empty count
  def server_init(self,names, sizes):
    for name,size in zip(names,sizes):
      #print(str(name)+" "+str(size))
      self.servers[name]=np.zeros(size,dtype=int)

#keep the count of memory location being used
  def server_update(self,name):
    pos=np.argmin(self.servers[name])
    self.servers[name][pos]=1
    return pos

#add pair< server, memory location> to a hash
  def add_hash(self,hash,server_memory):
    if self.hash_map.get(hash)==None :
      self.hash_map[hash]=[]
    self.hash_map[hash].append(server_memory)

### calculate divisors to find out the approprate list of servers, and number of server
  def get_server_list(self,n):
    divs = [1]
    for i in range(2,int(math.sqrt(n))+1):
        if n%i == 0:
          if i<=self.max_server:
            divs.extend([i])
          if n/i<=self.max_server:
            divs.extend([int(n/i)])
    if n<=self.max_server:
      divs.extend([n])
    divs=list(set(divs))
    divs.sort()

    server_list=[]
    for i in range(self.max_server):
      server_list.append(i)
    return server_list,divs[-1]

# Calculate the free storage size
  def server_occupency(self,names):
    free=[]
    for name in names:
      free.append(np.count_nonzero(self.servers[name]==0))
    return free


  def hash_delete(self):
    return 0  

  def location_of(self,hash):  #hash
    if self.hash_map.get(hash)==None :
      return 0;
    else:
      return self.hash_map[hash]

  def hash_of(self):  #location
    return;
#############                server end               ################


###################################################################################
#                         READ (blocks and string)                                #
###################################################################################
  def read_data(self,user_id,file_id):
    data=[]
    hashes=self.users[user_id][file_id]
    for hash in hashes:
      servers_loc=np.array(self.location_of(hash))
      
      tmp=[]
      for loc in servers_loc:
        tmp_dat=self.data_server[loc[0]].read(loc[1],1)
        if tmp_dat!= None:
          tmp = tmp+tmp_dat

      max_match =self.integrity_check(tmp,hash)
      if max_match !=0:
        data.append(max_match)
    str_data=''.join(data)
    return data, str_data


######################################################################
#                                WRITE                               #
######################################################################
  def write_data(self,data,user_id,file_id,block_size=64,redundency=3):
    
    if self.add_user_file(user_id,file_id) == 0:
      return self.update_data(data,user_id,file_id,block_size,redundency)
    blocks, hashes = self.block_create(data,block_size)
    server_list,num= self.get_server_list(redundency*len(blocks))
    #print(num)
    er=self.data_disperse(blocks, hashes,server_list,num,redundency)
    for hash in hashes:
      self.add_user_file_block(user_id,file_id,hash)
    
    return er



######################################################################
#                               UPDATE                               #
######################################################################
  def update_data(self,data,user_id,file_id,block_size=64,redundency=3):
    old_hashes=self.users[user_id][file_id]
    new_blocks, new_hashes = self.block_create(data,block_size)
    
    diff_hash = set(new_hashes)-set(old_hashes)
    index= [i for i, el in enumerate(new_hashes) if el in diff_hash]
    
    print("Changed in" + str(index))
    diff_blocks=[]
    for i in index:
        diff_blocks.append(new_blocks[i])
    diff_hash=list(diff_hash)
    self.users[user_id][file_id]= new_hashes
    server_list,num= self.get_server_list(redundency*len(diff_hash))
    er=self.data_disperse(diff_blocks, diff_hash,server_list,num,redundency)
    #print(num)
    return er




  def delete_data(self):
    return 0

#creates blocks and hash, returns both
  def block_create(self,data,block_size):
    blocks = [data[i:i+block_size] for i in range(0, len(data), block_size)]
    hashes=[]
    for block in blocks:
      hashes.append(self.hash_generation(block))
    return blocks,hashes

  def hash_generation(self,block):
    return sha256(block.encode('utf-8')).hexdigest()

#rating calculation just from duplication, other factors will be added
  def rating_calculation(self,subsets_hashes,server_list):
    server_num_all=len(server_list)
    taken=[]
    dupli =[]
    for subsets in subsets_hashes :
      rating=np.zeros(server_num_all,dtype=int)
      for hash in subsets:
        servers_loc=self.location_of(hash)
        if servers_loc != 0:
          for loc in servers_loc:
            rating[loc[0]]=rating[loc[0]]+1
      ##print(rating)
      
      dupli.append(rating)
      
      #greedy
      #if the server already taken we won't take it
      rating[taken] = -1
      server_id = np.argmax(rating)
      taken.append(server_id)
      ##print(server_id)
    #dp =solve (dp,0,len(subset,0,[],dupli))
    return taken

  def solve (self,dp,i,n, mask,taken,rating):
      #print(str(i)+" and "+str(mask)+str(taken))
      if i>=n:
        return 0
      if(dp[i][mask] != 0):
        return dp[i][mask];
      ans = 0;
      for k in range (n):
        if k not in taken:
          takenew = taken
          takenew.append(k)
          ans = max(ans,solve(i+1,k,takenew)+rating[i][mask])
      ans=ans+np.max(rating[i])
      dp[i][mask]=ans
      return ans

################################################
#############     data_disperse   ##############

  def data_disperse(self, blocks, hashes, server_list,server_num, redundency=3):
    #data_server= self.data_server
    block_tot =[]
    hash_tot =[]
    #redundency=3
    #server_list= [0,1,2,3,4,5]
    if int(len(blocks)*redundency)%server_num != 0:
      return "Error Number of Server is not Correct"

    #subset er lenth fixed
    subset_len= int(len(blocks)*redundency/server_num)

    #data redudency bar copy kora
    for i in range (redundency):
      block_tot=block_tot+blocks
      hash_tot=hash_tot+hashes

    #block niye subset banano
    subsets_block = [block_tot[i:i+subset_len] for i in range(0, len(block_tot), subset_len)]
    subsets_hashes = [hash_tot[i:i+subset_len] for i in range(0, len(hash_tot), subset_len)]

    #rating calculator kon server e pathaite hobe sei ID dey
    taken=self.rating_calculation(subsets_hashes,server_list)


    i=0
    for subsets,bls in zip(subsets_hashes,  subsets_block) :
      serv=taken[i]
      #serv is the dataserver ID

      #print("server"+str(serv))
      for hash,datas in zip(subsets,bls):
        servers_loc=np.array(self.location_of(hash))
        #print(servers_loc)
        if not (servers_loc.shape != () and [serv,-1]  in servers_loc):
          # eida on## print(' servers pathaite hobe '+ str(serv))
          #server memory list update
          pos = self.server_update(serv)
          #pos , position empty on that server
          self.data_server[serv].write(pos,datas)
          #hash list update
          self.add_hash(hash,[serv,pos])

      # eida on## print(str(i)+' subset sesh')
      i=i+1
    return "Success"



  def integrity_check(self,datas,hash):  #from Dataserver
    datas_clean = list(filter(lambda data: data is not None, datas))
    matched_data=max(set(datas_clean), key = datas_clean.count,default=None)
    if matched_data==None:
      return 0
    new_hash=self.hash_generation(matched_data)
    if new_hash != hash:
      return 0
    return matched_data

  
  def inconsistency_check(self): #among Index Server
    return 0

class MerkleTreeNode:
    def __init__(self,value):
        self.left = None
        self.right = None
        self.value = value
        self.hashValue = hashlib.sha256(value.encode('utf-8')).hexdigest()
    
def buildTree(leaves):
    nodes = []
    for i in leaves:
        nodes.append(MerkleTreeNode(i))

    while len(nodes)!=1:
        temp = []
        for i in range(0,len(nodes),2):
            node1 = nodes[i]
            if i+1 < len(nodes):
                node2 = nodes[i+1]
            else:
                temp.append(nodes[i])
                break
            #print("Left child : "+ node1.value + " | Hash : " + node1.hashValue +" \n")
            #print("Right child : "+ node2.value + " | Hash : " + node2.hashValue +" \n")
            concatenatedHash = node1.hashValue + node2.hashValue
            parent = MerkleTreeNode(concatenatedHash)
            parent.left = node1
            parent.right = node2
            #print("Parent(concatenation of *"+ node1.value + "* and *" + node2.value + "*) : " +parent.value + " | Hash : " + parent.hashValue +" \n")
            temp.append(parent)
        nodes = temp 
    return nodes

def check_match(node1, node2):
  if node1 == None:
    return 
  if node1.value==node2.value:
    return node1.value
  else:
    x = check_match(node1.left,node2.left)
    y = check_match(node1.right,node2.right)
  if x !=None and y!=None: 
    return x,y
  if x !=None: 
    return x
  if y !=None: 
    return x


In [30]:
class client:
  def __init__(self,id):
    self.id=id
    self.en_key=None
    self.dn_key=None
    self.bs = AES.block_size
    self.iv= b'\x02\xd3\xefg\x05\xf6\xa6\x9cK\xe5\xfaJ\xc1@\xa7 '
    # initial vector (iv) used to make different cypher for same msg

  #mode en,dn, both
  def take_key(self, key,mode='en'):
    if mode== 'en':
      self.en_key=hashlib.sha256(key.encode()).digest()
    elif mode == 'dn':
      self.dn_key=hashlib.sha256(key.encode()).digest()
    else:
      self.dn_key=hashlib.sha256(key.encode()).digest()
      self.en_key=hashlib.sha256(key.encode()).digest()


  def encrypt(self, raw):
    raw = self._pad(raw)
    #iv = Random.new().read(AES.block_size)
    iv=self.iv
    cipher = AES.new(self.en_key, AES.MODE_CBC, iv)
    return base64.b64encode(iv + cipher.encrypt(raw.encode()))


  def decrypt(self, enc):
      enc = base64.b64decode(enc)
      #iv = enc[:AES.block_size]
      iv= self.iv
      cipher = AES.new(self.dn_key, AES.MODE_CBC, iv)
      return self._unpad(cipher.decrypt(enc[AES.block_size:])).decode('utf-8')

  def _pad(self, s):
      return s + (self.bs - len(s) % self.bs) * chr(self.bs - len(s) % self.bs)

  @staticmethod
  def _unpad(s):
      return s[:-ord(s[len(s)-1:])]



  def read(self,fileid,server):
    bl,data = server.read_data(self.id,fileid)
    return bl,self.decrypt(data)

  def write(self,data,fileid,server,blocksize=64):
    data= self.encrypt(data).decode("utf-8") 
    return server.write_data(data,self.id,fileid,blocksize)

  def update(self,data,fileid,server,blocksize=64):
    data= self.encrypt(data);
    return server.write_data(data,self.id,fileid,blocksize)

  def delete(self):
    return 0
  def create_blocks(self,data,server,blocksize=64):
    data= self.encrypt(data).decode("utf-8") 
    blocks,hashes = server.block_create(data,blocksize)
    return blocks


## **Initialization** ☑

In [31]:
ds=[]
names=[]
sizes=[]
for i in range(1000):
  ds.append(data_server(i,1000))
  names.append(i)
  sizes.append(1000)

In [32]:
boss = index_server(0,ds)

In [33]:
boss.server_init(names,sizes)

File Size Here

In [23]:
data = ''.join(random.choice(string.ascii_letters) for i in range(16*1024*1024))

Run only to change number of maximum server for user

In [34]:
boss.max_server = 80

Dont Run If **Large**

In [ ]:
data

# Direct Query to Index Server (For Test)

In [11]:
prev=time.time()
boss.write_data(data,'u0','file0',block_size=8*1024,redundency=4)
cur= time.time()
print(cur-prev)


0.23383164405822754


In [13]:
data=data.replace('k','S',5)

In [12]:
prev=time.time()
boss.read_data('u0','file0')
cur= time.time()
print(cur-prev)

0.1059885025024414


In [14]:
prev=time.time()
boss.update_data(data,'u0','file0',8*1024)
cur= time.time()
print(cur-prev)

Changed in[0]
0.06813859939575195


In [15]:
_,read_dat=boss.read_data('u0','file0')
len(read_dat)

16777216

([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], 8)

# For user Call ▶

In [35]:
user0=client('u0')
user0.take_key('$a33|12',mode='both')

In [36]:
data = "Lorem ipsum dolor sit amet, consectetur adipiscing elit. Curabitur sed luctus ex, id porttitor felis. Suspendisse scelerisque risus ex. Nulla quis augue in metus tincidunt scelerisque. Cras ultricies dolor ipsum, ac faucibus magna vulputate eget. Curabitur bibendum felis eros, quis congue felis blandit in. Pellentesque in est a mauris eleifend sollicitudin. Curabitur a rutrum ex, vitae volutpat nibh. Class aptent taciti sociosqu ad litora torquent per conubia nostra, per inceptos himenaeos. Lorem ipsum dolor sit amet, consectetur adipiscing elit. Vestibulum sed augue et erat dapibus dapibus. Sed vel suscipit mauris. Proin libero nulla, congue sed commodo sit amet, auctor eget velit. Curabitur varius, sem sit amet dapibus finibus, enim nisl scelerisque ipsum, vel convallis nulla dolor non magna. Sed mollis mauris sit amet eros consequat, nec maximus lorem iaculis. Sed non tellus convallis, dictum nisl in, aliquam est. Etiam ac nibh mattis, dictum odio nec, porttitor eros. Fusce egestas, odio non egestas aenean."

In [37]:
user0.write(data,'file0',boss)

'Success'

In [38]:
read_blocks,str_data = user0.read('file0',boss)
str_data

'Lorem ipsum dolor sit amet, consectetur adipiscing elit. Curabitur sed luctus ex, id porttitor felis. Suspendisse scelerisque risus ex. Nulla quis augue in metus tincidunt scelerisque. Cras ultricies dolor ipsum, ac faucibus magna vulputate eget. Curabitur bibendum felis eros, quis congue felis blandit in. Pellentesque in est a mauris eleifend sollicitudin. Curabitur a rutrum ex, vitae volutpat nibh. Class aptent taciti sociosqu ad litora torquent per conubia nostra, per inceptos himenaeos. Lorem ipsum dolor sit amet, consectetur adipiscing elit. Vestibulum sed augue et erat dapibus dapibus. Sed vel suscipit mauris. Proin libero nulla, congue sed commodo sit amet, auctor eget velit. Curabitur varius, sem sit amet dapibus finibus, enim nisl scelerisque ipsum, vel convallis nulla dolor non magna. Sed mollis mauris sit amet eros consequat, nec maximus lorem iaculis. Sed non tellus convallis, dictum nisl in, aliquam est. Etiam ac nibh mattis, dictum odio nec, porttitor eros. Fusce egestas

In [39]:
old_blocks= user0.create_blocks(data,boss)

In [40]:
tree_old = buildTree(old_blocks)
tree_new = buildTree(read_blocks)

In [41]:
check_match(tree_old[0],tree_new[0])

'4b90e60fe2c938de30113d21406152c42dc520acfc65dcb06ee3a85326414c576fcbecc31465394d71664a065823d09b8dab7dd4b5fb20a4a0c27db9d346ae8e'

Unavailable Data server

In [ ]:
result=[]

In [ ]:
num_off= 80
#avail = np.random.randint(15,size=num_off)
avail = np.arange(num_off-1,-1,-1,dtype=int)
avail

array([79, 78, 77, 76, 75, 74, 73, 72, 71, 70, 69, 68, 67, 66, 65, 64, 63,
       62, 61, 60, 59, 58, 57, 56, 55, 54, 53, 52, 51, 50, 49, 48, 47, 46,
       45, 44, 43, 42, 41, 40, 39, 38, 37, 36, 35, 34, 33, 32, 31, 30, 29,
       28, 27, 26, 25, 24, 23, 22, 21, 20, 19, 18, 17, 16, 15, 14, 13, 12,
       11, 10,  9,  8,  7,  6,  5,  4,  3,  2,  1,  0])

In [ ]:
for i  in range(num_off):
  ds[avail[i]].available = False
  _,str_data = boss.read_data('u0','file0')
  print("After Removing "+str(i+1)+" server, "+str(len(str_data)/len(data)*100)+"% data recovered")
  result.append([i+1,len(str_data)/len(data)*100])
  print('.................................\n\n')
for i  in range(num_off):
  ds[avail[i]].available = True

In [ ]:
pd.DataFrame.from_dict(result).to_csv('dup/results16MB_8KB_80_RF5.csv',index=False)

# **For Performance Test Run from here**

In [ ]:
# Google Drive Mount
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/Colab Notebooks/
%ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks
'Copy of deduplication Final.ipynb'  'dup (1)'@
'Copy of deduplication.ipynb'         ICT_MS_DEDUPLICATION.ipynb
 deduplication.ipynb                 'MS_Copy of Copy of deduplication.ipynb'
 dup@                                 Untitled0.ipynb


In [ ]:
r=8
lf=1024*1024
lb = 1024
redun=np.arange(3,r+1,dtype=int)
max_server = np.array([10,20,40,80])
InitSize = np.array([  1 , 4, 8, 16,32,64])
file_size=np.array(InitSize*lf)
InitSize = np.array([  2 , 4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048, 4*1024, 8*1024 ])
block_size = np.array(InitSize*lb)

In [ ]:
file_size

array([ 1048576,  4194304,  8388608, 16777216, 33554432, 67108864])

In [ ]:
block_size

array([   2048,    4096,    8192,   16384,   32768,   65536,  131072,
        262144,  524288, 1048576, 2097152, 4194304, 8388608])

In [ ]:
redun

array([3, 4, 5, 6, 7, 8])

In [ ]:
res_time_tot=[]
res_ft_tot=[]

In [ ]:
datas={}
for f in file_size:
 data = ''.join(random.choice(string.ascii_letters) for i in range(f))
 datas[str(f)]=data

In [ ]:
for m in max_server:
  for r in redun:
    for f in file_size:
      data=datas[str(f)]
      for b in block_size:
        if b> f:
          break;
        res_time=[]
        res_time.append(m)
        res_time.append(r)
        res_time.append(f)
        res_time.append(b)
        
        ds=[]
        names=[]
        sizes=[]
        for i in range(120):
          ds.append(data_server(i,1000))
          names.append(i)
          sizes.append(1000)
        boss = index_server(0,ds)
        boss.server_init(names,sizes)
        boss.max_server = m

        prev=time.time()
        boss.write_data(data,'u0','file0',block_size=b,redundency=r)
        cur= time.time()
        res_time.append(cur-prev)

        prev=time.time()
        read_blocks,str_data = boss.read_data('u0','file0')
        cur= time.time()
        res_time.append(cur-prev)

        ##Fault Tollarance##
        avail = np.arange(m-1,-1,-1,dtype=int)
        res_ft=[]
        res_ft.append(m)
        res_ft.append(r)
        res_ft.append(f)
        res_ft.append(b)
        for i  in range(m):
          ds[avail[i]].available = False
          _,str_data = boss.read_data('u0','file0')
          #print("After Removing "+str(i+1)+" server, "+str(len(str_data)/len(data)*100)+"% data recovered")
          res_ft.append(len(str_data)/len(data)*100)
          #print('.................................\n\n')
        for i  in range(m):
          ds[avail[i]].available = True
        
        #MHT
        prev=time.time()
        tree_old = buildTree(read_blocks)
        tree_new = buildTree(read_blocks)
        check_match(tree_old[0],tree_new[0])
        cur= time.time()
        res_time.append(cur-prev)

        res_time_tot.append(res_time)
        res_ft_tot.append(res_ft)

In [ ]:
res_time_df = pd.DataFrame(res_time_tot,columns=['max_server','Redundancy','file_size','block_size','write_time','read_time','MHT Time'])
res_ft_df = pd.DataFrame(res_ft_tot,)


In [ ]:
res_time_df.to_csv('dup/result-read-write3.csv',index=False)
res_ft_df.to_csv('dup/result-fault3.csv',index=False)